# Notebook Instructions
<i>You can run the notebook document sequentially (one cell a time) by pressing <b> shift + enter</b>. While a cell is running, a [*] will display on the left. When it has been run, a number will display indicating the order in which it was run in the notebook [8].</i>

<i>Enter edit mode by pressing <b>`Enter`</b> or using the mouse to click on a cell's editor area. Edit mode is indicated by a green cell border and a prompt showing in the editor area.</i>

# Delta Hedging

Delta is the change in the price of an option for a unit change in the price of the underlying asset. With the changes in the price of the underlying, the value of the portfolio changes. Hence, Delta hedging is done to protect the portfolio against any changes in the price of the underlying asset.

## Methodology

The steps required to make the portfolio of a long call option Delta neutral are as follows:

5. Determine the Delta of the call option
6. Buy or sell the futures to make Delta close to zero

## Import the libraries

In [1]:
# Data manipulation
import numpy as np
import pandas as pd
import datetime

# To calculate Greeks
import mibian

## Read data

Here we collect the  data from <a href="https://nseindia.com/products/content/derivatives/equities/historical_fo.htm">nseindia</a> and save it as a CSV File. 

Then we do the following tasks:
1. Use pandas to import the CSV file as a data frame
1. Only keep data of the 10000 strike call and columns which are relevant
2. Only keep top 2 rows of data to simplify and learn the process of Delta hedge

In [2]:
def read_data(inst_name):
    opt = pd.read_csv(inst_name  + ".csv")            
    opt = opt[(opt['Option Type'] == 'CE') & (opt['Strike Price'] == 10000)]        
    opt = opt[['Symbol','Date','Expiry','Option Type','Strike Price','Close','futures_price']]         
    return opt

Nifty_Opt = read_data('../data_modules/NIFTY_GS_data')
Nifty_Opt = Nifty_Opt[:2]
Nifty_Opt.head()

,Symbol,Date,Expiry,Option Type,Strike Price,Close,futures_price
3281,NIFTY,03-Oct-17,28-Dec-17,CE,10000,184.20,9934.00
3406,NIFTY,04-Oct-17,28-Dec-17,CE,10000,208.95,10004.25


## Calculate time to expiry

We convert the trading date ('Date') and expiry date ('Expiry') into datetime object and then, calculate and store the time to expiry in days. 

In [3]:
Nifty_Opt.Expiry = pd.to_datetime(Nifty_Opt.Expiry)
Nifty_Opt.Date = pd.to_datetime(Nifty_Opt.Date)
Nifty_Opt['time_to_expiry']= (Nifty_Opt.Expiry - Nifty_Opt.Date).dt.days            
Nifty_Opt.head()    

,Symbol,Date,Expiry,Option Type,Strike Price,Close,futures_price,time_to_expiry
3281,NIFTY,2017-10-03,2017-12-28,CE,10000,184.20,9934.00,86
3406,NIFTY,2017-10-04,2017-12-28,CE,10000,208.95,10004.25,85


The time to expiry on 03 October and 04 October is 86 and 85 days respectively.

## Compute implied volatility and Delta

The implied volatility is computed using the Black Scholes (BS) function from the mibian library. The BS function requires five input parameters, the current value of the stock or index, the strike price, the risk-free rate, the time to expiry, and the option price to return the implied volatility. 

Then, we will use implied volatility as the input to compute the Delta of the call option.

In [4]:
def implied_volatility_options(opt):
    opt['IV'] = np.nan        
    for i in range(0,len(opt)):                 
        opt.iloc[i,opt.columns.get_loc('IV')] = mibian.BS([opt.iloc[i]['futures_price'], 
                                                               opt.iloc[i]['Strike Price'], 
                                                               0, 
                                                               opt.iloc[i]['time_to_expiry']], 
                                                              callPrice=opt.iloc[i]['Close']
                                                             ).impliedVolatility        
    return opt  

Nifty_Opt = implied_volatility_options(Nifty_Opt)    
Nifty_Opt.head()

,Symbol,Date,Expiry,Option Type,Strike Price,Close,futures_price,time_to_expiry,IV
3281,NIFTY,2017-10-03,2017-12-28,CE,10000,184.20,9934.00,86,11.169434
3406,NIFTY,2017-10-04,2017-12-28,CE,10000,208.95,10004.25,85,10.741711


In [5]:
def delta_options(opt):
    opt['delta'] = np.nan    
    for i in range(0,len(opt)):         
        if opt.iloc[i]['Option Type'] == 'CE':            
            opt.iloc[i,opt.columns.get_loc('delta')] = mibian.BS([opt.iloc[i]['futures_price'], 
                                                               opt.iloc[i]['Strike Price'], 
                                                               0, 
                                                               opt.iloc[i]['time_to_expiry']], 
                                                              volatility=opt.iloc[i]['IV']
                                                             ).callDelta        
    return opt  

Nifty_Opt = delta_options(Nifty_Opt)    
Nifty_Opt

,Symbol,Date,Expiry,Option Type,Strike Price,Close,futures_price,time_to_expiry,IV,delta
3281,NIFTY,2017-10-03,2017-12-28,CE,10000,184.20,9934.00,86,11.169434,0.462146
3406,NIFTY,2017-10-04,2017-12-28,CE,10000,208.95,10004.25,85,10.741711,0.513607


The Delta for 10000 strike call is 0.46 on 03 October. We will multiply this Delta value with the lot size of Nifty (75) to get the total Delta value.

In [6]:
Nifty_Lot_Size = 75
Nifty_Opt['total_delta'] = Nifty_Opt['delta'] * Nifty_Lot_Size
Nifty_Opt

,Symbol,Date,Expiry,Option Type,Strike Price,Close,futures_price,time_to_expiry,IV,delta,total_delta
3281,NIFTY,2017-10-03,2017-12-28,CE,10000,184.20,9934.00,86,11.169434,0.462146,34.660955
3406,NIFTY,2017-10-04,2017-12-28,CE,10000,208.95,10004.25,85,10.741711,0.513607,38.520558


## Delta hedging with Nifty futures

We will determine the number of Nifty futures to buy to make the portfolio of call options Delta neutral. The total Delta of the call options is 34.66. Thus, we would sell 34.66 shares of Nifty futures to Delta hedge the portfolio of call options. Since we can't sell any arbitrary number of share, we will round the number of Nifty futures required to Delta hedge to a multiple of 5.

<i>Note: Nifty futures trade in the multiples of 75, therefore, practically we should round the futures to a multiple of 75.</i>

In [7]:
Nifty_Opt['Fut_Quantity'] = -Nifty_Opt.total_delta // 5 * 5
Nifty_Opt

,Symbol,Date,Expiry,Option Type,Strike Price,Close,futures_price,time_to_expiry,IV,delta,total_delta,Fut_Quantity
3281,NIFTY,2017-10-03,2017-12-28,CE,10000,184.20,9934.00,86,11.169434,0.462146,34.660955,-35.0
3406,NIFTY,2017-10-04,2017-12-28,CE,10000,208.95,10004.25,85,10.741711,0.513607,38.520558,-40.0


We have to sell 35 shares of Nifty futures to make the portfolio Delta neutral. Therefore, the portfolio has the following positions:
1. Long 75, Nifty 10000 strike call 
2. Short 35, Nifty futures

## Futures PnL

The futures PnL is computed by taking the difference between the futures price and multiplying it by the number of the futures contract.

In [8]:
futures_pnl = (Nifty_Opt.futures_price - Nifty_Opt.futures_price.shift(1)) * Nifty_Opt.Fut_Quantity.shift(1)
futures_pnl

3281        NaN
3406   -2458.75
dtype: float64

## Call PnL

Similar to futures PnL, the call PnL is computed by taking the difference between the call option price data and multiplying with the Nifty lot size.

In [9]:
call_pnl = (Nifty_Opt.Close - Nifty_Opt.Close.shift(1))  * Nifty_Lot_Size
call_pnl

3281        NaN
3406    1856.25
Name: Close, dtype: float64

## Portfolio PnL

The portfolio PnL is computed by add call and futures PnL.

In [10]:
call_pnl + futures_pnl

3281      NaN
3406   -602.5
dtype: float64

The profit on call option is approximately INR 1860 and loss on the Nifty futures is approximately INR 2460. Thus, the portfolio PnL is loss of approximately INR 600. The loss of INR 600 is due to the effect of other Greeks such as Gamma and Theta.
Coming up, further in this section, how we can exploit Delta neutrality to gain profits by Gamma scalping.
<BR>